## Awesome encryption Scheme
This challenge give me [encrypting code](encryptor.py) and [ciphertext](encrypted)

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from hashlib import md5
from os import urandom
from flag import flag

keys = [md5(urandom(3)).digest() for _ in range(2)]


def bytexor(da,ta): return bytes(i^j for i,j in zip(da,ta))


def get_ciphers(iv1, iv2):
    return [
        AES.new(keys[0], mode=AES.MODE_CBC, iv=iv1),
        AES.new(keys[1], mode=AES.MODE_CFB, iv=iv2, segment_size=8*16),
    ]

def encrypt(m: bytes, iv1: bytes, iv2: bytes) -> bytes:
    m = pad(m,32)
    ciphers = get_ciphers(iv1, iv2)
    c = m
    for cipher in ciphers:
        c = b''.join(i[16:]+bytexor(i[:16],cipher.encrypt(i[16:])) for i in [c[i:i+32] for i in range(0,len(c),32)])
    return c

plaintext = f'finally now i am able to send my secret with double security and double trust, {flag}'.encode()
iv1, iv2 = urandom(16),urandom(16)

ciphertext = encrypt(plaintext, iv1, iv2)
ciphertext = b":".join([x.hex().encode() for x in [iv1, iv2, ciphertext]])

open('encrypted','wb').write(ciphertext)

Here I have 2 keys generated from md5 hash of 2 3-byte strings. Plaintext is encrypted AES twice, first with mode CBC and second with CFB. Mode CBC and CFB can be easily found on internet, for example [here](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation). From here I can summarise the process encryption:

- Split the plaintext to blocks of 32 bytes (**c[i:i+32]**)
- Each time of encryption, the program above:
+ Remain 16 last bytes (**i[16:]**)
+ Encrypt 16 last bytes (**cipher.encrypt(i[16:])**) and xor received ciphertext with 16 first bytes

If I call the 16 first bytes of plaintext $P_a$, 16 second bytes of plaintext $P_b$, first AES encryption $E_1$ (with **keys[0]**) and second AES encryption $E_2$ (with **keys[1]**), then after the first encryption: $$C_a || C_b = P_b || (P_a \oplus E_1(P_b \oplus iv_1))$$ (here || means concatration of strings)

Let $P_a \oplus E_1(P_b \oplus iv_1) = T$ Then the second encryption: $$C'_a || C'_b = T || (C_a \oplus T \oplus E_2(iv_2))$$

So $T$ is first 16 bytes of ciphertext, and I also knew $P_a$, $P_b$ and $iv_1$, so I can bruteforce the key of $E_1$

In [4]:
from Crypto.Cipher import AES
from tqdm import tqdm
from hashlib import md5
from binascii import unhexlify

def bytexor(da,ta): 
    return bytes(i^j for i,j in zip(da,ta))

with open("encrypted", "rb") as f:
    data = f.read().split(b":")

iv1, iv2, ciphertext = [unhexlify(i) for i in data]
plaintext = b'finally now i am able to send my secret with double security and double trust, '
# keys = [b'\x0e\x98\xd7\xa85\xb5\x8d\xd53\x8e\x11\xb3*\xad\x8fX', b'\x192\xe2\xb9k\x83\x13\x883\xfa\xc3HZt\xa1\xe5']
# ct = encrypt(plaintext[:32], iv1, iv2)
# print(decrypt(ciphertext, iv1, iv2))
p1a = plaintext[:16]
p1b = plaintext[16:32]
t1 = ciphertext[:16]
for i in tqdm(range(2**24)):
    key = md5(i.to_bytes(3, 'big')).digest()
    c1 = AES.new(key, AES.MODE_ECB).encrypt(bytexor(p1b, iv1))
    if bytexor(c1, p1a) == t1:
        print(key)
        break

 60%|█████▉    | 10020252/16777216 [05:02<03:24, 33116.86it/s]

b'\x0e\x98\xd7\xa85\xb5\x8d\xd53\x8e\x11\xb3*\xad\x8fX'


I also knew $C_a \oplus T \oplus E_2(iv_2)$ (next 16 bytes of $T$), and notice that $C_a = P_b$ :)) Then I can easily recover the second key

In [6]:
p1a = plaintext[:16]
p1b = plaintext[16:32]
t1 = ciphertext[:16]
t1_ = ciphertext[16:32]
c1_ = bytexor(t1_, p1b)
for i in tqdm(range(2**24)):
    key = md5(i.to_bytes(3, 'big')).digest()
    c1 = AES.new(key, AES.MODE_ECB).encrypt(iv2)
    if bytexor(c1, c1_) == t1:
        print(key)
        break

  6%|▌         | 924898/16777216 [00:25<07:11, 36775.01it/s]

b'\x192\xe2\xb9k\x83\x13\x883\xfa\xc3HZt\xa1\xe5'


Okey I got 2 keys, then I can write decryption function

In [7]:
def decrypt(c: bytes, iv1: bytes, iv2: bytes) -> bytes:
    ciphers = get_ciphers(iv1, iv2)
    m = c
    for cipher in ciphers[::-1]:
        ct = b''
        for i in range(0, len(c), 32):
            ctt = m[i:i+32]
            second = ctt[:16]
            first = bytexor(ctt[16:], cipher.encrypt(second))
            ct = ct + first + second
        m = ct
    return m

Here only encryption function is used because I reuse the way challenge split the plaintext, and then recover full plaintext

In [9]:
def get_ciphers(iv1, iv2):
    return [
        AES.new(keys[0], mode=AES.MODE_CBC, iv=iv1),
        AES.new(keys[1], mode=AES.MODE_CFB, iv=iv2, segment_size=8*16),
    ]

keys = [b'\x0e\x98\xd7\xa85\xb5\x8d\xd53\x8e\x11\xb3*\xad\x8fX', b'\x192\xe2\xb9k\x83\x13\x883\xfa\xc3HZt\xa1\xe5']
print(decrypt(ciphertext, iv1, iv2))

b'finally now i am able to send my secret with double security and double trust, shaktictf{Well now I know that it is not an awesome encryption scheme}\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'


Thanks for reading!